### This script is to calculate the joint distribution of (x,y) of make/miss a FG from a location.

In [ ]:
%%shell
apt-get update -qq > /dev/null
apt-get install openjdk-8-jdk-headless -qq > /dev/null
wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
tar xf spark-2.4.7-bin-hadoop2.7.tgz
pip install -q findspark

In [ ]:
%%shell
pip install --upgrade pip
pip install lxml
git clone https://github.com/pmcwhannel/NBA-analytics.git
mv NBA-analytics NBAanalytics # So importing functions is easy

     |████████████████████████████████| 1.5MB 4.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Cloning into 'NBA-analytics'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 8), reused 16 (delta 3), pack-reused 0
Unpacking objects: 100% (26/26), done.


Import functions and test them out

In [ ]:
# Have to rename drive to get rid of NBA-analytics -> NBAanalytics
from NBAanalytics.ShotChartFunctions import *
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext(appName="YourTest", master="local[*]")

In [ ]:
test_link = 'https://www.basketball-reference.com/boxscores/199911020CHH.html' # Designed strongly based on this link
get_single_shot_chart_data(test_link)[:1] # Return is list of lists

Time to create RDD boxscore URLS

In [ ]:
# boxscore_URLS = sc.textFile("NBAanalytics/box_score_links.txt") 
# Preprocess text file to reduce to smaller subset. grab all from 2000
# ~1200s box scores per year (this is two halves of two seasons)
lines = open("NBAanalytics/box_score_links.txt").readlines()
subset_URLS = []
for line in lines:
  if int(line[47:51]) == 2000:
    subset_URLS.append(line.strip())
  else:
    continue

In [ ]:
boxscore_URLS = sc.parallelize(subset_URLS) # create subset to test on
boxscore_URLS.count()

1281

In [ ]:
import timeit as tt
start= tt.default_timer()
print(boxscore_URLS.map(lambda x: get_single_shot_chart_data(x)).take(1))
stop = tt.default_timer()
print('It took {} seconds to extract all shot chart data.'.format(stop - start))

[[['amaecjo01', 1, 'make', '143', '269', '11:34.0', '10', '2-0', 'Orlando', 'ORL', 'MIA', '2000'], ['amaecjo01', 1, 'make', '169', '288', '10:40.0', '13', '6-5', 'Orlando', 'ORL', 'MIA', '2000'], ['amaecjo01', 1, 'miss', '253', '245', '10:11.0', '20', '6-5', 'Orlando', 'ORL', 'MIA', '2000'], ['abdulta01', 1, 'miss', '229', '340', '9:48.0', '20', '6-5', 'Orlando', 'ORL', 'MIA', '2000'], ['abdulta01', 1, 'make', '55', '389', '9:19.0', '15', '8-5', 'Orlando', 'ORL', 'MIA', '2000'], ['abdulta01', 1, 'miss', '118', '252', '8:47.0', '6', '8-7', 'Orlando', 'ORL', 'MIA', '2000'], ['armstda01', 1, 'miss', '66', '318', '7:52.0', '7', '8-7', 'Orlando', 'ORL', 'MIA', '2000'], ['abdulta01', 1, 'miss', '156', '426', '7:31.0', '21', '8-9', 'Orlando', 'ORL', 'MIA', '2000'], ['wallabe01', 1, 'miss', '113', '284', '7:00.0', '7', '8-11', 'Orlando', 'ORL', 'MIA', '2000'], ['willimo01', 1, 'miss', '107', '288', '6:31.0', '7', '8-14', 'Orlando', 'ORL', 'MIA', '2000'], ['abdulta01', 1, 'miss', '169', '299', 

In [ ]:
# limit data to subset and do all testing on that then will increase scale and see how performance changes.

[]